In [1]:
import asyncio

from autogen_core.model_context import BufferedChatCompletionContext
from autogen_ext.models.openai import OpenAIChatCompletionClient

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console

In [2]:
model_client = OpenAIChatCompletionClient(model="gpt-4o")
model_context = BufferedChatCompletionContext(buffer_size=5)

async def lookup_hotel(location: str) -> str:
    return f"Here are some hotels in {location}: hotel1, hotel2, hotel3."

async def lookup_flight(origin: str, destination: str) -> str:
    return f"Here are some flights from {origin} to {destination}: flight1, flight2, flight3."

async def book_trip() -> str:
    return "Your trip is booked!"

In [3]:
travel_advisor = AssistantAgent(
    "Travel_Advisor",
    model_client,
    tools=[book_trip],
    description="Helps with travel planning.",
)
hotel_agent = AssistantAgent(
    "Hotel_Agent",
    model_client,
    tools=[lookup_hotel],
    description="Helps with hotel booking.",
)
flight_agent = AssistantAgent(
    "Flight_Agent",
    model_client,
    tools=[lookup_flight],
    description="Helps with flight booking.",
)
termination = TextMentionTermination("TERMINATE")
team = SelectorGroupChat(
    [travel_advisor, hotel_agent, flight_agent],
    model_client=model_client,
    termination_condition=termination,
    model_context=model_context,
)

In [4]:
await Console(team.run_stream(task="Book a 3-day trip to new york."))

---------- TextMessage (user) ----------
Book a 3-day trip to new york.
---------- ToolCallRequestEvent (Travel_Advisor) ----------
[FunctionCall(id='call_NUTeHkSF8nnx4qKi0wYWbRHJ', arguments='{}', name='book_trip')]
---------- ToolCallExecutionEvent (Travel_Advisor) ----------
[FunctionExecutionResult(content='Your trip is booked!', name='book_trip', call_id='call_NUTeHkSF8nnx4qKi0wYWbRHJ', is_error=False)]
---------- ToolCallSummaryMessage (Travel_Advisor) ----------
Your trip is booked!
---------- TextMessage (Hotel_Agent) ----------
I can help you with finding hotels and accommodations in New York for your 3-day trip. Would you like that?
---------- TextMessage (Flight_Agent) ----------
Have you booked the flight for your trip to New York? If not, I can help you find available flights. Let me know your current city, and I'll get the details started for you!
---------- ToolCallRequestEvent (Hotel_Agent) ----------
[FunctionCall(id='call_AczNlkMs5Z95iJCuXt2dnf92', arguments='{"locati

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Book a 3-day trip to new york.', type='TextMessage'), ToolCallRequestEvent(source='Travel_Advisor', models_usage=RequestUsage(prompt_tokens=66, completion_tokens=10), metadata={}, content=[FunctionCall(id='call_NUTeHkSF8nnx4qKi0wYWbRHJ', arguments='{}', name='book_trip')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Travel_Advisor', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='Your trip is booked!', name='book_trip', call_id='call_NUTeHkSF8nnx4qKi0wYWbRHJ', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='Travel_Advisor', models_usage=None, metadata={}, content='Your trip is booked!', type='ToolCallSummaryMessage'), TextMessage(source='Hotel_Agent', models_usage=RequestUsage(prompt_tokens=92, completion_tokens=25), metadata={}, content='I can help you with finding hotels and accommodations in New York for your 3-day trip.